In [ ]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
from tqdm import tqdm
import json
from sklearn.preprocessing import StandardScaler
import joblib

# PyTorch imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

# Configuration
DATA_DIR = "/kaggle/input/MABe-mouse-behavior-detection"
MODEL_PATH = "/kaggle/working/behavior_model.pth"
BATCH_SIZE = 1024
LEARNING_RATE = 0.001
EPOCHS = 25
HIDDEN_SIZE = 512

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define the neural network model
class BehaviorNet(nn.Module):
    def __init__(self, input_size, num_classes):
        super(BehaviorNet, self).__init__()
        self.fc1 = nn.Linear(input_size, HIDDEN_SIZE)
        self.bn1 = nn.BatchNorm1d(HIDDEN_SIZE)
        self.fc2 = nn.Linear(HIDDEN_SIZE, HIDDEN_SIZE // 2)
        self.bn2 = nn.BatchNorm1d(HIDDEN_SIZE // 2)
        self.fc3 = nn.Linear(HIDDEN_SIZE // 2, HIDDEN_SIZE // 4)
        self.bn3 = nn.BatchNorm1d(HIDDEN_SIZE // 4)
        self.fc4 = nn.Linear(HIDDEN_SIZE // 4, num_classes)
        self.dropout = nn.Dropout(0.3)
        
    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = F.relu(self.bn3(self.fc3(x)))
        x = self.dropout(x)
        x = self.fc4(x)
        return x

# Fixed Dataset class - keep data on CPU, move to GPU in training loop
class MouseBehaviorDataset(Dataset):
    def __init__(self, features, labels):
        # Keep data on CPU, move to GPU in training loop
        self.features = torch.from_numpy(features).float()
        self.labels = torch.from_numpy(labels).float()
        
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

# Load the pre-extracted features
def load_preprocessed_data():
    """Load pre-extracted features and labels"""
    print("Loading preprocessed data...")
    
    # Assuming you've already extracted features and saved them
    # For now, we'll create dummy data based on your extracted shapes
    features_shape = (880844, 5)
    labels_shape = (880844, 20)
    
    # Create dummy data (replace with your actual feature loading code)
    features = np.random.randn(*features_shape).astype(np.float32)
    labels = np.random.rand(*labels_shape).astype(np.float32)
    
    # Apply some threshold to make it more like real labels
    labels = (labels > 0.8).astype(np.float32)
    
    print(f"Loaded data shape: {features.shape}, Labels shape: {labels.shape}")
    return features, labels

# GPU-accelerated training
def train_model():
    """Train a behavior classification model with GPU acceleration"""
    # Load preprocessed data
    features, labels = load_preprocessed_data()
    
    if len(features) == 0:
        print("No training data available. Using rule-based approach for prediction.")
        return None, None
    
    # Scale features
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)
    
    # Split data
    train_size = int(0.8 * len(features_scaled))
    train_features, val_features = features_scaled[:train_size], features_scaled[train_size:]
    train_labels, val_labels = labels[:train_size], labels[train_size:]
    
    # Create datasets and dataloaders
    train_dataset = MouseBehaviorDataset(train_features, train_labels)
    val_dataset = MouseBehaviorDataset(val_features, val_labels)
    
    # Use DataLoader with fewer workers to avoid CUDA issues
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, 
                              num_workers=0, pin_memory=True)  # Set num_workers=0 to avoid CUDA issues
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, 
                            num_workers=0, pin_memory=True)
    
    # Create model
    input_size = features.shape[1]
    num_classes = labels.shape[1]
    model = BehaviorNet(input_size, num_classes).to(device)
    
    # Define loss and optimizer
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-5)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, factor=0.5)
    
    # Training loop
    best_val_loss = float('inf')
    
    for epoch in range(EPOCHS):
        # Training phase
        model.train()
        train_loss = 0.0
        
        for batch_idx, (data, target) in enumerate(train_loader):
            # Move data to GPU
            data, target = data.to(device), target.to(device)
            
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            
            if batch_idx % 100 == 0:
                print(f"Epoch {epoch}, Batch {batch_idx}, Loss: {loss.item():.4f}")
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for data, target in val_loader:
                # Move data to GPU
                data, target = data.to(device), target.to(device)
                output = model(data)
                loss = criterion(output, target)
                val_loss += loss.item()
        
        # Calculate average losses
        train_loss /= len(train_loader)
        val_loss /= len(val_loader)
        
        # Update learning rate
        scheduler.step(val_loss)
        
        print(f"Epoch {epoch}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")
        
        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), MODEL_PATH)
            print(f"Saved best model with validation loss: {val_loss:.4f}")
    
    # Save scaler
    joblib.dump(scaler, "/kaggle/working/scaler.joblib")
    print(f"Model training complete. Best validation loss: {best_val_loss:.4f}")
    
    return model, scaler

# GPU-accelerated inference
def process_test_video(lab_id, video_id, model, scaler):
    """Process a single test video and predict behaviors with GPU acceleration"""
    try:
        # Load test tracking data
        tracking_path = f"{DATA_DIR}/test_tracking/{lab_id}/{video_id}.parquet"
        tracking_data = pd.read_parquet(tracking_path)
        
        # Extract features (simplified version)
        features = extract_simple_features(tracking_data)
        
        if len(features) == 0:
            return []
        
        # Scale features
        features_scaled = scaler.transform(features)
        
        # Move to GPU and predict
        with torch.no_grad():
            model.eval()
            features_tensor = torch.from_numpy(features_scaled).float().to(device)
            pred = model(features_tensor)
            pred_probs = torch.sigmoid(pred).cpu().numpy()
        
        # Get frame numbers
        frames = tracking_data['video_frame'].unique()
        frames = frames[:len(features)]  # Ensure same length as features
        
        # Convert predictions to behavior events
        frame_predictions = []
        threshold = 0.5  # Prediction threshold
        
        for i, (frame, probs) in enumerate(zip(frames, pred_probs)):
            for class_idx, prob in enumerate(probs):
                if prob > threshold:
                    # For each mouse pair (simplified)
                    frame_predictions.append({
                        'video_id': video_id,
                        'frame': frame,
                        'agent_id': 'mouse1',  # Simplified
                        'target_id': 'mouse2',  # Simplified
                        'action': f"behavior_{class_idx}"  # Simplified behavior name
                    })
        
        # Group consecutive frames with the same behavior
        predictions_df = pd.DataFrame(frame_predictions)
        if len(predictions_df) == 0:
            return []
        
        # Sort by frame
        predictions_df = predictions_df.sort_values('frame')
        
        # Group consecutive frames with same behavior, agent, and target
        predictions_df['group'] = (
            (predictions_df['agent_id'] != predictions_df['agent_id'].shift()) |
            (predictions_df['target_id'] != predictions_df['target_id'].shift()) |
            (predictions_df['action'] != predictions_df['action'].shift()) |
            (predictions_df['frame'] != predictions_df['frame'].shift() + 1)
        ).cumsum()
        
        # Aggregate into start and stop frames
        grouped = predictions_df.groupby(['video_id', 'agent_id', 'target_id', 'action', 'group'])
        result = grouped.agg(
            start_frame=('frame', 'min'),
            stop_frame=('frame', 'max')
        ).reset_index().drop('group', axis=1)
        
        return result.to_dict('records')
        
    except Exception as e:
        print(f"Error processing test video {lab_id}/{video_id}: {e}")
        return []

# Simple feature extraction for testing
def extract_simple_features(tracking_data):
    """Simple feature extraction for testing"""
    features = []
    
    # Group by frame
    frames = tracking_data['video_frame'].unique()
    
    for frame in frames:
        frame_data = tracking_data[tracking_data['video_frame'] == frame]
        
        # Get mouse positions
        mice_data = {}
        for mouse in frame_data['mouse_id'].unique():
            mouse_df = frame_data[frame_data['mouse_id'] == mouse]
            # Use body center if available, otherwise average all points
            if 'body_center' in mouse_df['bodypart'].values:
                center = mouse_df[mouse_df['bodypart'] == 'body_center'][['x', 'y']].mean()
            else:
                center = mouse_df[['x', 'y']].mean()
            mice_data[mouse] = center
        
        # Skip frames with insufficient mice
        if len(mice_data) < 2:
            continue
            
        # Extract simple features (just positions of two mice)
        frame_features = []
        mouse_ids = sorted(mice_data.keys())[:2]  # Just use first two mice
        
        # Add mouse positions
        for mouse in mouse_ids:
            pos = mice_data[mouse]
            frame_features.extend([pos['x'], pos['y']])
        
        # Add distance between mice
        if len(mouse_ids) >= 2:
            pos1 = mice_data[mouse_ids[0]]
            pos2 = mice_data[mouse_ids[1]]
            dist = np.sqrt((pos1['x'] - pos2['x'])**2 + (pos1['y'] - pos2['y'])**2)
            frame_features.append(dist)
        
        features.append(frame_features)
    
    return np.array(features, dtype=np.float32)

def main():
    # Check if test data is available
    test_csv_path = f"{DATA_DIR}/test.csv"
    test_tracking_dir = f"{DATA_DIR}/test_tracking"
    
    if os.path.exists(test_csv_path) and os.path.exists(test_tracking_dir):
        print("Processing test data...")
        
        # Load test metadata
        test_meta = pd.read_csv(test_csv_path)
        
        # Train a new model
        print("Training a new model...")
        model, scaler = train_model()
        
        if model is None:
            # Fall back to rule-based approach if training failed
            print("Using rule-based fallback...")
            all_predictions = []
            for _, row in tqdm(test_meta.iterrows(), total=len(test_meta)):
                lab_id = row['lab_id']
                video_id = row['video_id']
                
                # Simple rule-based approach
                try:
                    tracking_path = f"{DATA_DIR}/test_tracking/{lab_id}/{video_id}.parquet"
                    tracking_data = pd.read_parquet(tracking_path)
                    
                    mice_data = {}
                    for mouse in tracking_data['mouse_id'].unique():
                        mouse_df = tracking_data[tracking_data['mouse_id'] == mouse]
                        if 'body_center' in mouse_df['bodypart'].values:
                            center = mouse_df[mouse_df['bodypart'] == 'body_center'][['x', 'y']].mean()
                            mice_data[mouse] = center
                    
                    # Simple rule: if mice are close, predict "sniff"
                    mice_list = list(mice_data.keys())
                    for frame in tracking_data['video_frame'].unique():
                        for i, mouse1 in enumerate(mice_list):
                            for j, mouse2 in enumerate(mice_list):
                                if i != j:  # Only between different mice
                                    pos1 = mice_data[mouse1]
                                    pos2 = mice_data[mouse2]
                                    dist = np.sqrt((pos1['x'] - pos2['x'])**2 + (pos1['y'] - pos2['y'])**2)
                                    
                                    if dist < 50:  # Threshold for closeness
                                        all_predictions.append({
                                            'video_id': video_id,
                                            'frame': frame,
                                            'agent_id': mouse1,
                                            'target_id': mouse2,
                                            'action': 'sniff'
                                        })
                except Exception as e:
                    print(f"Error processing {lab_id}/{video_id}: {e}")
                    continue
            
            # Group consecutive frames
            predictions_df = pd.DataFrame(all_predictions)
            if len(predictions_df) > 0:
                predictions_df['group'] = (
                    (predictions_df['agent_id'] != predictions_df['agent_id'].shift()) |
                    (predictions_df['target_id'] != predictions_df['target_id'].shift()) |
                    (predictions_df['action'] != predictions_df['action'].shift()) |
                    (predictions_df['frame'] != predictions_df['frame'].shift() + 1)
                ).cumsum()
                
                grouped = predictions_df.groupby(['video_id', 'agent_id', 'target_id', 'action', 'group'])
                result = grouped.agg(
                    start_frame=('frame', 'min'),
                    stop_frame=('frame', 'max')
                ).reset_index().drop('group', axis=1)
                
                submission_df = result
            else:
                submission_df = pd.DataFrame(columns=['video_id', 'agent_id', 'target_id', 'action', 'start_frame', 'stop_frame'])
            
            submission_df['row_id'] = range(len(submission_df))
            submission_df = submission_df[['row_id', 'video_id', 'agent_id', 'target_id', 'action', 'start_frame', 'stop_frame']]
            submission_df.to_csv('/kaggle/working/submission.csv', index=False)
            print("Submission saved to /kaggle/working/submission.csv")
            return
        
        # Process each test video with the trained model
        all_predictions = []
        for _, row in tqdm(test_meta.iterrows(), total=len(test_meta)):
            lab_id = row['lab_id']
            video_id = row['video_id']
            
            predictions = process_test_video(lab_id, video_id, model, scaler)
            all_predictions.extend(predictions)
        
        # Create submission
        submission_df = pd.DataFrame(all_predictions)
        if len(submission_df) > 0:
            submission_df['row_id'] = range(len(submission_df))
            submission_df = submission_df[['row_id', 'video_id', 'agent_id', 'target_id', 'action', 'start_frame', 'stop_frame']]
        else:
            # Create empty submission with correct columns
            submission_df = pd.DataFrame(columns=['row_id', 'video_id', 'agent_id', 'target_id', 'action', 'start_frame', 'stop_frame'])
        
        # Save submission
        submission_df.to_csv('/kaggle/working/submission.csv', index=False)
        print("Submission saved to /kaggle/working/submission.csv")
        
    else:
        # Train model only
        print("No test data available, training model...")
        train_model()
        print("Model training complete.")

if __name__ == "__main__":
    main()

In [ ]:
import pandas as pd 
df = pd.read_csv("/kaggle/working/submission.csv")
df